In [2]:
"""
DOES NOT WORK : print is good, but the code after is not handled (BLUE, RED, etc) !!
"""
import common_immunogit

import asyncio
from playwright.async_api import async_playwright
import nest_asyncio
from bs4 import BeautifulSoup
import re
import os
import json
BLUE = '\033[94m'
RED = '\033[91m'
RESET = '\033[0m'

Structure : Done.


In [4]:
nest_asyncio.apply()

"""
ABOVE CODE SHOULD GO IN common_immunogit.py
"""
def create_query(domain="biomodels", offset=0, num_results=100):
    query_parts_full = {
        'mode': '*:*',
        'species': 'TAXONOMY:9606',
        'curation_status': 'curationstatus:"Manually curated"',
        'formats': 'modelformat:"SBML"',
        'kw': 'submitter_keywords:"Immuno-oncology"'
    }

    query_parts = [value for value in query_parts_full.values() if value]
    query = " AND ".join(query_parts)

    query_for_url = query.replace(" ", "%20").replace(":", "%3A").replace('"', "%22")
    url = f"https://www.ebi.ac.uk/biomodels/search?query={query_for_url}&domain={domain}&offset={offset}&numResults={num_results}"

    return query, url

def extract_ids(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            soup = BeautifulSoup(file, "html.parser")

        pattern = r"(BIOMD\d{10}|MODEL\d{10})"
        ids = set()

        for element in soup.find_all(string=True):
            content = element.strip()
            matches = re.findall(pattern, content)
            ids.update(matches)

        ids = sorted(ids)
        return ids
    except Exception as e:
        print(f"Error : {e}")
        return []

async def scrape_page(offset=0, num_results=100):
    query, url = create_query(domain="biomodels", offset=offset, num_results=num_results)
    output_file = "webpage_source.html"

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url)
        await page.wait_for_load_state("networkidle")
        page_source = await page.content()
        with open(output_file, "w", encoding="utf-8") as file:
            file.write(page_source)

        print(f"{BLUE}File saved : {output_file}{RESET}")
        await browser.close()

    ids = extract_ids(output_file)

    if os.path.exists(output_file):
        os.remove(output_file)
        print(f"{BLUE}File deleted: {output_file}{RESET}")
    return ids

async def main():
    offset = 0
    num_results = 100
    ids_agreg = []

    while True:
        print(f"{BLUE}Current offset : + {offset}{RESET}")
        ids = await scrape_page(offset, num_results)

        if not ids:
            print(f"{BLUE}Stopping : no more IDs.{RESET}")
            break

        ids_agreg.extend(ids)
        print(f"{BLUE}Stopping : collected {len(ids)} IDs, below 100 limit.{RESET}")
        print(f"{BLUE}Info : {len(ids_agreg)} IDs found.{RESET}")

        if len(ids) < num_results:
            break

        offset += num_results

    with open("/Users/guillaume.souede/PycharmProjects/immunogit/src/ids_agreg.json", "w") as f:
        json.dump(ids_agreg, f)

    return ids_agreg

    print(f"Collected IDs : {ids_agreg}")
    print(f"{BLUE}Number of Collected IDs : {len(ids_agreg)}{RESET}")
    print(f"{url}")

asyncio.run(main())

Current offset : + 0
File saved : webpage_source.html
File deleted: webpage_source.html
Stopping : collected 68 IDs, below 100 limit.
Info : 68 IDs found.


['BIOMD0000000741',
 'BIOMD0000000742',
 'BIOMD0000000743',
 'BIOMD0000000744',
 'BIOMD0000000746',
 'BIOMD0000000748',
 'BIOMD0000000749',
 'BIOMD0000000751',
 'BIOMD0000000752',
 'BIOMD0000000753',
 'BIOMD0000000754',
 'BIOMD0000000756',
 'BIOMD0000000757',
 'BIOMD0000000758',
 'BIOMD0000000759',
 'BIOMD0000000760',
 'BIOMD0000000764',
 'BIOMD0000000766',
 'BIOMD0000000767',
 'BIOMD0000000768',
 'BIOMD0000000769',
 'BIOMD0000000770',
 'BIOMD0000000778',
 'BIOMD0000000780',
 'BIOMD0000000781',
 'BIOMD0000000782',
 'BIOMD0000000791',
 'BIOMD0000000798',
 'BIOMD0000000801',
 'BIOMD0000000802',
 'BIOMD0000000812',
 'BIOMD0000000813',
 'BIOMD0000000877',
 'BIOMD0000000879',
 'BIOMD0000000880',
 'BIOMD0000000885',
 'BIOMD0000000886',
 'BIOMD0000000888',
 'BIOMD0000000891',
 'BIOMD0000000894',
 'BIOMD0000000900',
 'BIOMD0000000904',
 'BIOMD0000000908',
 'BIOMD0000000909',
 'BIOMD0000000910',
 'BIOMD0000000911',
 'BIOMD0000000912',
 'BIOMD0000000913',
 'BIOMD0000000919',
 'BIOMD0000000921',
